In [1]:
# CSVファイルに記載したフォルダ・PDFファイル名を読み込み、
# PDFファイルのテキストの属性情報をinfoフォルダに格納

import pandas as pd
import numpy as np
import MeCab
import fitz
import re
import sys
sys.path.append('/work_dir') 
import talknize_module_20240909 as tk
import csv
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
import logging


# ログファイルの設定
log_filename = "data/logfile_attrib_info.log"
logging.basicConfig(
    level=logging.INFO,  # ログレベルの設定 (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(levelname)s - %(message)s",  # ログのフォーマット
    handlers=[
        logging.FileHandler(log_filename, mode="a", encoding="utf-8"),  # ファイルに記録
        #logging.StreamHandler()  # コンソールにも出力
    ]
)


In [2]:
def extract_pdf_attributes(pdf_path, page_number, output_csv_path):
    """
    指定したPDFファイルのブロック・ライン・スパン単位でデータ属性を取得し、CSV形式で出力。
    
    Args:
        pdf_path (str): PDFファイルのパス。
        page_number (int): ページ番号（1から始まる）。
        output_csv_path (str): 出力するCSVファイルのパス。
        
    Returns:
        pd.DataFrame: 取得したデータ属性を保持するデータフレーム。
    """
    try:
        # PDFを開く
        doc = fitz.open(pdf_path)
        
        # ページ番号が正しいか確認
        if page_number < 1 or page_number > len(doc):
            print(f"エラー: ページ番号が総ページ数{len(doc)}を超えています。最終ページを対象に処理を進めます。")
            logging.warning(f"ページ番号が総ページ数{len(doc)}を超えています。最終ページを対象に処理を進めます。")
            page_number = len(doc)
        
        # 対象ページを取得（0インデックス）
        page = doc[page_number - 1]
        
        # ページ内のブロックを取得
        blocks = page.get_text("dict")["blocks"]
        
        # データ保持用リスト
        data = []
        
        for block_no, block in enumerate(blocks, start=1):
            if "lines" not in block:
                continue  # ブロックに行がない場合はスキップ
            
            for line_no, line in enumerate(block["lines"], start=1):
                if "spans" not in line:
                    continue  # 行にスパンがない場合はスキップ
                
                for span_no, span in enumerate(line["spans"], start=1):
                    # 各スパンの属性を取得
                    text = span.get("text", "")
                    bbox = line["bbox"]
                    font = span.get("font", "")
                    size = span.get("size", "")
                    color = span.get("color", "")
                    text_type = span.get("flags", "")
                    origin = span.get("origin", (None, None))  # originを取得
                    ascender = span.get("ascender", None)      # ascenderを取得
                    
                    # データをリストに追加
                    data.append({
                        "page_no": page_number,
                        "block_no": block_no,
                        "line_no": line_no,
                        "span_no": span_no,
                        "text": text,
                        "origin_x": origin[0],
                        "origin_y": origin[1],
                        "bbox_x0": bbox[0],
                        "bbox_y0": bbox[1],
                        "bbox_x1": bbox[2],
                        "bbox_y1": bbox[3],
                        "font": font,
                        "size": size,
                        "color": color,
                        "text_type": text_type,
                        "ascender": ascender,
                    })
        
        # PDFを閉じる
        doc.close()
        
        # pandasデータフレームに変換
        df = pd.DataFrame(data)
        
        # CSVとして出力
        df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
        #print(f"ターゲットページの属性情報がCSVに出力されました: {output_csv_path}")
        logging.info(f"ターゲットページの属性情報がCSVに出力されました: {output_csv_path}")
        return df
 
    except Exception as e:
        print(f"extract_pdf_attributesでエラーが発生しました: {e}")
        logging.error(f"extract_pdf_attributesでエラーが発生しました: {e}")
        return None


In [3]:
# CSVを読み込む関数
def load_csv(file_path):
    """
    CSVを読み込み、DataFrameを返す
    file_path: CSVファイルのパス
    """
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: CSV file not found at {file_path}")
        logging.error(f"Error: CSV file not found at {file_path}")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file at {file_path} is empty or invalid")
        logging.error(f"Error: CSV file at {file_path} is empty or invalid")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        logging.error(f"Error reading CSV file: {e}")
        return None

In [4]:
#全頁からブロックの属性情報を抽出・データフレームに格納

def extract_blocks_from_pdf(pdf_path):
    pdf_doc = fitz.open(pdf_path)
    data = []

    # 各ページからデータ抽出
    for page_num in range(len(pdf_doc)):
        page = pdf_doc[page_num]
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            block_bbox = block["bbox"]
            for line in block.get("lines", []):
                line_text = " ".join([span["text"] for span in line["spans"]])
                line_bbox = line["bbox"]
                data.append({
                    "page": page_num + 1,
                    "x0": line_bbox[0],
                    "y0": line_bbox[1],
                    "x1": line_bbox[2],
                    "y1": line_bbox[3],
                    "text": line_text.strip()
                })
    pdf_doc.close()
    return pd.DataFrame(data)

In [7]:
#データフレームに格納された（全ページの）属性情報からヘッダ・フッタ候補を抽出、同一内容のものを集約し件数をカウント

def identify_header_footer(data, threshold=40):
    # ページごとの y0 値の分布を取得
    headers = []
    footers = []

    for page_num, group in data.groupby("page"):
        y0_values = group["y0"].values
        if len(y0_values) == 0:
            continue
        y1_values = group["y1"].values
        if len(y1_values) == 0:
            continue

        min_y0, max_y0 = np.min(y0_values), np.max(y0_values)
        min_y1, max_y1 = np.min(y1_values), np.max(y1_values)

        # ヘッダ候補: y1 が分布の下位 (小さい値)
        header_candidates = group[np.abs(group["y1"] - min_y1) < threshold]
        headers.append(header_candidates)

        # フッタ候補: y0 が分布の上位 (大きい値)
        footer_candidates = group[np.abs(group["y0"] - max_y0) < threshold]
        footers.append(footer_candidates)

    # ヘッダ・フッタを統合
    headers = pd.concat(headers, ignore_index=True)
    footers = pd.concat(footers, ignore_index=True)

    # 頻出パターンを確認 (x0, y0, text を基準にグループ化)
    header_patterns = headers.groupby(["x1", "y1", "text"]).size().reset_index(name="count")
    footer_patterns = footers.groupby(["x0", "y0", "text"]).size().reset_index(name="count")

    return header_patterns, footer_patterns


In [10]:
# メインルーチン

def main():
    # CSVファイルのパス
    #csv_path = 'data/Attrib_Analysis_Target_20250114_test.csv'
    #csv_path = 'data/Attrib_Analysis_Target_20250218.csv'
    #csv_path = 'data/Attrib_Analysis_Target_20250218b.csv'
    csv_path = 'data/Attrib_Analysis_Target_20250219_Denka.csv'  # CSVのパスを指定
    

    # CSVを読み込む
    df_target = load_csv(csv_path)
    if df_target is None:
        print("Failed to load CSV. Exiting program.")
        logging.error("Failed to load CSV. Exiting program.")
        return

    # 必要な列名を指定
    folder_col = '格納フォルダ'
    file_col = '統合報告書PDF'
    target_col = 'target_page'

    # CSVに必要な列がなければエラー表示
    if folder_col not in df_target.columns or file_col not in df_target.columns or target_col not in df_target.columns:
        print(f"Error: CSV file must contain '{folder_col}' and '{file_col}' and '{target_page}' columns")
        logging.error(f"Error: CSV file must contain '{folder_col}' and '{file_col}' and '{target_page}' columns")
        return

    # リストの企業ごとに、ターゲットページ（のみ）の属性情報を取得・処理・保存
    #file_paths = []
    for index, row in df_target.iterrows():

        #================================================================
        #前処理（フォルダの確認・パス名の定義　等）
        folder_name = 'ir/2025/' + row[folder_col]
        file_name = row[file_col]
        target_page = row[target_col]

        # パスが有効かチェック
        #　file_pathを作成してfile_pathsにリストとして追記・保存するが、必要情報がない場合は''としておく
        if pd.isna(folder_name) or pd.isna(file_name):
            print(f"Skipping row {index}: Folder or File is missing")
            logging.warning(f"Skipping row {index}: Folder or File is missing")
            file_path =''
            return

        print(f"==== Processing {folder_name}/{file_name} ====")
        logging.info(f"==== Processing {folder_name}/{file_name} ====")

        #属性情報を格納するフォルダ（info）の有無を確認、なければ作成
        info_file_path = folder_name + '/' + 'info'
        if not os.path.exists(info_file_path):
            os.makedirs(info_file_path)

        #================================================================
        #属性情報の保存・データフレームへの格納
        # file_path（対象PDF）を生成 
        file_path = folder_name + '/' + file_name
        #file_path = os.path.join(folder_name, file_name)
        #file_paths.append(file_path)

        #属性情報を格納するcsvファイルのパスを作成
        output_file_path = info_file_path + '/' +file_name[:len(file_name)-4] + "_attrib_page" + str(target_page) + ".csv"

        # 属性情報を抽出し、CSVfファイルに出力の上、以降の作業用にデータフレームdf_attribに格納
        df_attrib = extract_pdf_attributes(file_path, target_page, output_file_path)
        #print(df_attrib)
    
        #================================================================
        #テキストボックスの始点座標のプロット（ブロック番号付き）
        # line_no=1 のデータを抽出
        plot_data = df_attrib

        # bbox_x0 と bbox_y0 をプロット (Y軸を上下反転)
        plt.figure(figsize=(10, 6))
        plt.scatter(plot_data['bbox_x0'], plot_data['bbox_y0'], alpha=0.7, s=10)
        
        # block_no を各点に表示
        for x, y, block_no in zip(plot_data['bbox_x0'], plot_data['bbox_y0'], plot_data['block_no']):
            plt.text(x, y, str(block_no), fontsize=8, ha='right', va='bottom')  # block_no をラベルとして表示
        
        plt.title('Scatter Plot of box_x0 vs bbox_y0 with block_no', fontsize=14)
        plt.xlabel('bbox_x0', fontsize=12)
        plt.ylabel('bbox_y0 (Inverted)', fontsize=12)
        plt.gca().invert_yaxis()  # Y軸を上下反転
        plt.grid(alpha=0.3)
        
        # プロットを保存
        plot_file_path = info_file_path + '/' +file_name[:len(file_name)-4] + '_' +str(target_page) + ".png"
        # DPIを設定し保存
        plt.savefig(plot_file_path, dpi=300, bbox_inches='tight')  
        # プロットを表示
        #plt.show()
        # プロットを閉じる（これにより画面に表示されない）
        plt.close()
        #print(f"プロットが保存されました: {plot_file_path}")
        logging.info(f"プロットが保存されました: {plot_file_path}")

        #================================================================
        #　上下左右のマージン検討・設定時に参照するブロックのxy座標及びテキスト情報をファイルに格納

        n_write = 40
        print_flag = False

        # ヘッダ候補（y1）・フッタ候補（y0）.・左マージン候補（x0）・右マージン候補（x1）を作成
        sorted_y1 = df_attrib.sort_values(by='bbox_y1', ascending=True)[['block_no', 'bbox_y1', 'text']]
        sorted_y0 = df_attrib.sort_values(by='bbox_y0', ascending=False)[['block_no', 'bbox_y0', 'text']]
        sorted_x1 = df_attrib.sort_values(by='bbox_x1', ascending=True)[['block_no', 'bbox_x1', 'text']]
        sorted_x0 = df_attrib.sort_values(by='bbox_x0', ascending=False)[['block_no', 'bbox_x0', 'text']]

        #画面表示用
        if print_flag == True:
            print("=== bbox(y1) for Header ===")
            print(sorted_y1)
            print()
            print("=== bbox(y0) for Footer ===")
            print(sorted_y0)
            print()
            print("=== bbox(x0) for Left Margin ===")
            print(sorted_x0)
            print()
            print("=== bbox(x0) for Right Margin ===")
            print(sorted_x1)
        
        output_file_path = info_file_path  + "/" + "Margin_Candidate_list_page-" + str(target_page) + ".txt"
        
        # ヘッダ候補（y1）・フッタ候補（y0）.・左マージン候補（x0）・右マージン候補（x1）をテキストファイルに書き出す
        with open(output_file_path, "w", encoding="utf-8") as file:
            file.write("=== bbox(y1) for Header ===\n")
            for _, row in sorted_y1.head(n_write).iterrows():
                # 'text'列を左寄せに整形して保存
                file.write(f"{row['block_no']}\t{row['bbox_y1']:.2f}\t{row['text']}\n")
            file.write("\n\n")  
            file.write("=== bbox(y0) for Footer ===\n")
            for _, row in sorted_y0.head(n_write).iterrows():
                # 'text'列を左寄せに整形して保存
                file.write(f"{row['block_no']}\t{row['bbox_y0']:.2f}\t{row['text']}\n")
            file.write("\n\n")  
            file.write("=== bbox(x0) for Left Margin ===\n")
            for _, row in sorted_x1.head(n_write).iterrows():
                # 'text'列を左寄せに整形して保存
                file.write(f"{row['block_no']}\t{row['bbox_x1']:.2f}\t{row['text']}\n")
            file.write("\n\n")  
            file.write("=== bbox(x0) for Right Margin ===\n")
            for _, row in sorted_x0.head(n_write).iterrows():
                # 'text'列を左寄せに整形して保存
                file.write(f"{row['block_no']}\t{row['bbox_x0']:.2f}\t{row['text']}\n")

        #=============================================
        #以降はターゲットページのみではなく、各社の全頁からデータ取得、候補情報を集計
        #全頁を読み込み、処理するため、かなり負荷がかかるはず
        # PDFデータを抽出
        df_pdf_all_pages = extract_blocks_from_pdf(file_path)
    
        # ヘッダ・フッタを識別
        headers, footers = identify_header_footer(df_pdf_all_pages)
    
        # 結果をソートして表示
        headers_sorted = headers.sort_values("y1", ascending=True)
        footers_sorted = footers.sort_values("y0", ascending=True)
    
        #print("Identified Header Patterns:")
        #print(headers_sorted)
        #print()
        #print("Identified Footer Patterns:")
        #print(footers_sorted)
    
        # ソート済み結果をCSVに保存
        output_header_path = info_file_path + "/" + "header_candidates.csv"
        output_footer_path = info_file_path + "/" + "footer_candidates.csv"
        headers_sorted.to_csv(output_header_path, index=False, encoding="utf-8-sig")
        footers_sorted.to_csv(output_footer_path, index=False, encoding="utf-8-sig")
        #print(f"ヘッダ・フッタ候補の統計情報が保存されました: header_candidates.csv, footer_candidates.csv")
        logging.info(f"ヘッダ・フッタ候補の統計情報が保存されました: header_candidates.csv, footer_candidates.csv")

        #=============================================
        #テキスト取得時に参照するmargin.csvファイルを作成
        #既に存在する場合はスキップ（書き込み情報の上書き防止のため）
         #margin.csvが存在しない場合には、ブランクのcsvを書き込み
        # チェックと生成
        margin_info_path = info_file_path + "/" + "margin.csv"
        if not os.path.exists(margin_info_path):
            # データを作成 (全てNone)
            margin_init = {
                "header_y": [None],
                "footer_y": [None],
                "left_margin_x": [None],
                "right_margin_x": [None]
            }
            df_margin = pd.DataFrame(margin_init)
            # CSVファイルを保存
            df_margin.to_csv(margin_info_path, index=False)
            #print(f"{margin_info_path} を新新規作成しました。")
            logging.info(f"{margin_info_path} を新規作成しました。")

        else:
            #print(f"{margin_info_path} は既に存在しています。")    
            logging.warning(f"{margin_info_path} は既に存在しています。")    

        #=============================================
        #テキスト取得時に参照する multicolumn.txtファイルを作成
        #既に存在する場合はスキップ（書き込み情報の上書き防止のため）
        # multicolumn.txtが存在しない場合には、ブランクのtxtを書き込み
        # チェックと生成
        multicolumn_info_path = info_file_path + "/" + "multicolumn.txt"
        if not os.path.exists(multicolumn_info_path):
            # 空のテキストファイルを作成
            with open(multicolumn_info_path,"w"):pass
            logging.info(f"{multicolumn_info_path} を新規作成しました。")
        else:
            logging.warning(f"{multicolumn_info_path} は既に存在しています。")    

        
        print(f"{file_path}の処理が終了しました。")
        
if __name__ == "__main__":
    main()

#順に企業コード・企業名・PDFファイル名を取り込み


==== Processing ir/2025/40610_デンカ/denkareport2024.pdf ====
ir/2025/40610_デンカ/denkareport2024.pdfの処理が終了しました。
